In [25]:
from importlib import reload
import mapreduce
reload(mapreduce)
from mapreduce import FilteringMapReduce, map_reduce_df
import pandas as pd
import utils
reload(utils)
from utils import get_date_index, get_days_in_range
import metric
reload(metric)
from metric import NWRMSLE_log, get_weights
import numpy as np

In [2]:
# fill with median from previous two weeks
days = get_days_in_range('2017-06-16', '2017-08-15')

In [5]:
cols = ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion']
types = {'id': 'int32',
         'date': 'int16',
         'item_nbr': 'int32',
         'store_nbr': 'int16',
         'unit_sales': 'float32',
         'onpromotion': bool}
mapreduce = FilteringMapReduce(lambda df: df[df.date.isin(days)], cols, types)
train = map_reduce_df('./data/train_processed.csv', mapreduce, types=types, position=110000000, cols=cols, verbose=True)

Reading from position 110000000, batch size 10000000...
Filtering 10000000...
Filtered 930790, mapping...
Mapped, reducing...
Done.
Reading from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 5497041, mapping...
Mapped, reducing...
Done.
End of dataset is found.


In [6]:
# filling empty dates
train.ix[train.unit_sales < 0, 'unit_sales'] = 0
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(["date", "store_nbr", "item_nbr"], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=["date", "store_nbr", "item_nbr"]
    )
)
train.loc[:, "unit_sales"].fillna(0, inplace=True)
train.loc[:, "onpromotion"].fillna("False", inplace=True)
train.reset_index(inplace=True)

In [9]:
# log
train['unit_sales'] = np.log1p(train['unit_sales'])

In [11]:
days_train1 = get_days_in_range('2017-06-16', '2017-06-30')
days_train2 = get_days_in_range('2017-07-01', '2017-07-15')
days_train3 = get_days_in_range('2017-07-16', '2017-07-31')
days_test = get_days_in_range('2017-08-01', '2017-08-15')

mean_train1 = train[train.date.isin(days_train1)]
mean_train2 = train[train.date.isin(days_train2)]
mean_train3 = train[train.date.isin(days_train3)]
mean_test = train[train.date.isin(days_test)]
del train

In [21]:
mean_train2.head()

,date,store_nbr,item_nbr,id,unit_sales,onpromotion
1474200,1642.0,46.0,1695836.0,120739967.0,3.221996,False
1474201,1642.0,46.0,1695837.0,120739968.0,2.211018,False
1474202,1642.0,46.0,1695840.0,120739969.0,1.609438,False
1474203,1642.0,46.0,1695845.0,120739970.0,3.850148,True
1474204,1642.0,46.0,1695846.0,120739971.0,1.384828,False


In [26]:
def ev(train, test):
    train = train.groupby(['item_nbr','store_nbr', 'onpromotion'], as_index=False).agg({'unit_sales':'mean'})
    train.rename(columns={'unit_sales':'unit_sales_mean'}, inplace=True)  
    test = test.merge(train, on=['item_nbr','store_nbr', 'onpromotion'], how='left')
    test.fillna(0.0, inplace=True)
    y_true = test['unit_sales']
    y_pred = test['unit_sales_mean']
    weights = get_weights(test['item_nbr'])
    error = NWRMSLE_log(y_true, y_pred, weights)
    return error

errors = []
errors.append(ev(mean_train1, mean_train2))
errors.append(ev(mean_train2, mean_train3))
errors.append(ev(mean_train3, mean_test))
np.mean(errors)

0.4635163813459644

In [27]:
errors

[0.4844975701263138, 0.4461081096833141, 0.45994346422826543]

In [28]:
test = pd.read_csv('./data/test.csv')
train = mean_test.groupby(by=['item_nbr','store_nbr', 'onpromotion'], as_index=False).agg({'unit_sales':'mean'})
test = test.merge(train, on=['item_nbr','store_nbr', 'onpromotion'], how='left')
test.fillna(0.0, inplace=True)
test.ix[test.unit_sales < 0, 'unit_sales'] = 0
test['unit_sales'] = np.expm1(test['unit_sales'])

In [29]:
#0.673
test[['id', 'unit_sales']].to_csv('./submissions/previous_week_log_mean_0.46.csv.gz', index=False, compression="gzip")

In [30]:
test.head()

,id,date,store_nbr,item_nbr,onpromotion,unit_sales
0,125497040,2017-08-16,1,96995,False,1.550849
1,125497041,2017-08-16,1,99197,False,1.620741
2,125497042,2017-08-16,1,103501,False,0.000000
3,125497043,2017-08-16,1,103520,False,1.846996
4,125497044,2017-08-16,1,103665,False,2.809150
